In [ ]:
import pandas as pd

# 📂 Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 📄 Ruta del archivo en Google Drive (ajustar según tu ubicación)
file_path = "/content/drive/MyDrive/Data_Augmentation/JUAN/AUDIO_DATA_AUGM/DATA/Gastos.xlsx"

# 📊 Cargar el archivo Excel
xls = pd.ExcelFile(file_path)
df = pd.read_excel(xls, sheet_name=xls.sheet_names[0])  # Cargar la primera hoja

# 🔍 Ver los primeros datos
print("📌 Datos originales:")
print(df.head())

# 🛠️ **Limpieza y Normalización** 🛠️

# 🔹 Quitar espacios en blanco en los nombres de las columnas
df.columns = df.columns.str.strip()

# 🔹 Normalizar texto en la columna 'Section' (convertir todo a mayúsculas)
df['Section'] = df['Section'].str.strip().str.upper()

# 🔹 Eliminar filas vacías o duplicadas
df = df.drop_duplicates().reset_index(drop=True)

# ✅ **Aplicar Primera Forma Normal (1FN)**
# Separamos los estudiantes en una tabla aparte

# Crear tabla de estudiantes dividiendo los nombres separados por comas
students_df = df[['Course #', 'Students']].copy()
students_df = students_df.assign(Students=students_df['Students'].str.split(', ')).explode('Students')
students_df = students_df.drop_duplicates().reset_index(drop=True)

# ✅ **Aplicar Segunda Forma Normal (2FN)**
# Dividir datos en tablas independientes según sus relaciones

# Tabla de cursos
cursos_df = df[['Course #', 'Course name']].drop_duplicates().reset_index(drop=True)

# Tabla de profesores
profesores_df = df[['Instructor']].drop_duplicates().reset_index(drop=True)

# Tabla de salones
salones_df = df[['Room', 'Seat Number']].drop_duplicates().reset_index(drop=True)

# Tabla de horarios
horarios_df = df[['Course #', 'Section', 'Room', 'Class Time', 'Meeting Days', 'Instructor']].drop_duplicates().reset_index(drop=True)

# ✅ **Aplicar Tercera Forma Normal (3FN)**
# La información de estudiantes está separada y no hay dependencias transitivas

# 📁 Guardar los archivos normalizados en Google Drive
output_path = "/content/drive/MyDrive/Data_Augmentation/JUAN/AUDIO_DATA_AUGM/DATA/Gastos_Norm01.xlsx"

with pd.ExcelWriter(output_path) as writer:
    df.to_excel(writer, sheet_name="Datos Originales", index=False)
    cursos_df.to_excel(writer, sheet_name="Cursos", index=False)
    profesores_df.to_excel(writer, sheet_name="Profesores", index=False)
    salones_df.to_excel(writer, sheet_name="Salones", index=False)
    horarios_df.to_excel(writer, sheet_name="Horarios", index=False)
    students_df.to_excel(writer, sheet_name="Estudiantes", index=False)

print(f"✅ Normalización completada. Archivo guardado en:\n {output_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📌 Datos originales:
    Section      Course #                Course name  Seat Number  \
0        AM     CSIS 1410       Computer Programming           33   
1        PM  Database Dev  Introduction to Databases           30   
2  Brighton      WLD 1120          Welding Processes           15   
3        am      ART 1130                  3D Design           33   
4        AM     CSIS 1410       Computer Programming           33   

   Number of Students  Room  Class Time Meeting Days  Instructor  \
0                  22   204   7:30-9:55            B  Henrichsen   
1                   5   202  11:30-1:55         Both       Magee   
2                  28   213   7:30-9:55            A   Humphries   
3                  14   505  11:30-1:55            B   Dr. Smith   
4                  22   204   7:30-9:55            B  Henrichsen   

                           